<a href="https://colab.research.google.com/github/proywm/PDC-concepts-LiveCoding/blob/main/LiveCoding_Concurrency_Simulating_Transportation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Introduction to Concurrency

Concurrency is a fundamental concept in computer science that allows multiple tasks to run simultaneously, or seemingly so, within a program. Understanding concurrency is crucial because it enables the efficient use of resources, improves performance, and is essential for developing responsive applications. In a world where computers are equipped with multiple cores, learning how to leverage concurrency can make a significant difference in the execution speed and responsiveness of programs.

In this tutorial, we will explore the concept of concurrency using a simple example: simulating a ride that transports passengers from one stop to another. We'll start by demonstrating how this process works sequentially—one step at a time—and then discuss how concurrency can improve the performance of this operation.



In [ ]:
!pip install matplotlib

##Sequential Execution

In the provided code, we simulate the process of a car (representing a bus) transporting passengers from a starting point to a destination. The key aspects of this sequential execution are:

1. **Single Task Handling**: The bus can only carry one passenger at a time (due to a bus capacity of 1). It must complete the full round trip—picking up a passenger, transporting them, and returning to the start—before it can pick up another passenger.
2. **Time Delay**: Each trip takes a certain amount of time, with a delay added to simulate the travel time of the bus. Since only one bus is active and carrying one passenger at a time, all other passengers must wait their turn.
3. **Sequential Order**: The process is strictly sequential. The bus must complete one trip entirely before starting the next, leading to a linear relationship between the number of passengers and the total time taken.


In [ ]:
import time
from IPython.display import clear_output

# Setup
total_passengers = 10
bus_capacity = 1
total_trips = 0
passengers_transported = 0
trip_details = []

# Track the position of the car
car_position = 0

def print_smileys(count, total_slots=10):
    smileys = "😃" * count + "⬜" * (total_slots - count)
    smiley_grid = []
    for i in range(2):  # Always print a 2x5 grid
        smiley_grid.append(smileys[i*5:(i+1)*5])
    return smiley_grid

# Transport function for a single car
def transport_passengers(car_emoji, lane):
    global total_trips, passengers_transported, car_position
    fixed_path_segment = "-" * 10  # Each segment between start, intermediate, and stop
    bus_paths = [
        f"{car_emoji}{fixed_path_segment}{fixed_path_segment}{fixed_path_segment}",
        f"{fixed_path_segment}{car_emoji}{fixed_path_segment}{fixed_path_segment}",
        f"{fixed_path_segment}{fixed_path_segment}{car_emoji}{fixed_path_segment}",
        f"{fixed_path_segment}{fixed_path_segment}{fixed_path_segment}{car_emoji}"
    ]

    while True:
        for path in bus_paths:
            if passengers_transported >= total_passengers:
                return
            car_position = bus_paths.index(path)
            visualize_trip(total_trips, passengers_transported)
            time.sleep(0.25)
        if passengers_transported < total_passengers:
            total_trips += 1
            passengers_transported += bus_capacity
            if passengers_transported > total_passengers:
                passengers_transported = total_passengers
            trip_details.append((total_trips, passengers_transported))
            car_position = len(bus_paths) - 1
            visualize_trip(total_trips, passengers_transported, final=True)
        time.sleep(0.25)  # Simulate the return of the bus
        for path in reversed(bus_paths):
            car_position = bus_paths.index(path)
            visualize_trip(total_trips, passengers_transported, returning=True)
            time.sleep(0.25)
        time.sleep(1)


# Visualization
def visualize_trip(trip_number, passengers_transported, final=False, returning=False):
    clear_output(wait=True)
    num_smileys_start = total_passengers - passengers_transported
    num_smileys_stop = passengers_transported
    start_smileys = print_smileys(num_smileys_start)
    stop_smileys = print_smileys(num_smileys_stop)

    fixed_path_segment = "-" * 10  # Each segment between start, intermediate, and stop
    car_path = ["🚗" + fixed_path_segment * 3, fixed_path_segment + "🚗" + fixed_path_segment * 2,
                fixed_path_segment * 2 + "🚗" + fixed_path_segment, fixed_path_segment * 3 + "🚗"]

    car_pos = car_path[car_position]

    print(f"Trip {trip_number}: {passengers_transported} passengers transported.")
    for row in range(2):
        if row == 0:
            print(start_smileys[row] + "  " + car_pos + "  " + stop_smileys[row])
        else:
            print(start_smileys[row] + "  " + " " * len(car_pos) + "  " + stop_smileys[row])

# Run the transport function
start_time = time.time()
transport_passengers("🚗", 0)
elapsed_time = time.time() - start_time
print(f"Total elapsed time: {elapsed_time:.2f} seconds")
throughput = passengers_transported / elapsed_time
print(f"Throughput: {throughput:.2f} passengers per second")
print()


Trip 10: 10 passengers transported.
⬜⬜⬜⬜⬜  🚗------------------------------  😃😃😃😃😃
⬜⬜⬜⬜⬜                                   😃😃😃😃😃
Total elapsed time: 32.71 seconds
Throughput: 0.31 passengers per second



### Demonstration of Sequential Execution

The code above is designed to visualize this sequential process:

- The `transport_passengers` function simulates a bus moving across a fixed path and picking up passengers one by one.
- The function iterates through each trip, moving the bus across the path, updating the number of passengers transported, and showing the state of the system.
- Each time the bus completes a trip and returns, the program checks if there are more passengers to transport, and if so, starts the next trip.

This approach highlights the key characteristics of sequential execution: only one task (one trip) is processed at a time, and the next task must wait until the previous one is fully completed.

### Performance Metrics

To measure the performance of this sequential approach, we introduce two key metrics:

1. **Elapsed Time**: This is the total time taken to transport all the passengers from start to destination. It includes the time spent during each trip as well as the idle time between trips.
   
   - **Measurement**: The elapsed time is recorded by capturing the time before starting the transportation and after all passengers have been transported. This gives us a clear measure of how long the entire process takes.

2. **Throughput**: Throughput measures the rate at which passengers are transported, typically represented as passengers per second.

   - **Measurement**: Throughput is calculated by dividing the total number of passengers transported by the elapsed time. A higher throughput indicates a more efficient process.

These metrics are critical in assessing the efficiency of our approach. In the context of concurrency, they help us understand how much time we save and how much more work we can accomplish when tasks are handled concurrently rather than sequentially.

### Summary

In this part of the tutorial, we have introduced the concept of concurrency and demonstrated sequential execution using a live coding example. We also introduced key performance metrics—elapsed time and throughput—that will help us quantify the efficiency of different approaches. In the following sections, we will explore how concurrency can be implemented to improve the performance of this passenger transportation simulation.

### Introduction to Concurrency with Multiple Buses

In the previous section, we explored sequential execution, where a single bus transported passengers from one stop to another, one by one. While this approach works, it is not efficient, especially when dealing with a large number of passengers. Now, we will introduce the concept of concurrency, which allows us to perform multiple tasks simultaneously, leading to faster and more efficient execution.

### Step 1: Introducing Multiple Buses

To begin with, let's think about how we might speed up the process of transporting passengers. One straightforward idea is to add more buses. Instead of just one bus making trips back and forth, we can have two buses working at the same time. This way, each bus can transport passengers independently, potentially halving the time it takes to transport everyone.

### Step 2: Creating Separate Tasks with Threads

To simulate the operation of two buses working simultaneously, we use a concept called *threads*. A thread is essentially a separate path of execution in your program. By using threads, we can have both buses running at the same time in our simulation.

In the code provided:

- We create two threads, each representing one of the buses (`🚗` and `🚙`).
- Each thread runs the same function, `transport_passengers`, but they operate independently of each other.

At this stage, it's important to note that each bus is still working with its own set of tasks—picking up passengers, transporting them, and returning for more. These tasks happen in parallel, thanks to the use of threads.

### Step 3: Managing Shared Resources

However, there is one important detail to consider: both buses are working towards the same goal of transporting all passengers. This means they share a common resource—the total number of passengers left to be transported. When multiple threads share data, it’s crucial to ensure that they don’t interfere with each other.

Without proper management, both buses might try to pick up the same passengers at the same time, leading to errors in our simulation. This situation is known as a *race condition*.

### Step 4: Synchronization with Locks

To prevent race conditions, we introduce a concept called *locks*. A lock ensures that only one thread can access the shared resource at a time. When one bus (or thread) is updating the number of passengers, the other bus has to wait its turn. This ensures that our data stays consistent and accurate.

In the code:

- A `lock` is used to synchronize access to the shared resource (`passengers_transported`).
- Each time a bus (thread) wants to update the number of passengers transported, it first acquires the lock. Once it’s done, it releases the lock so the other bus can proceed.

### Step 5: Visualizing Concurrent Execution

With these concepts in place—multiple threads, shared resources, and locks—we can now run our simulation:

- Two buses operate concurrently, transporting passengers simultaneously.
- The shared state (the number of passengers transported) is safely updated using locks.
- The visualization function (`visualize_trip`) shows the current state of the buses, allowing us to see how both buses are working together to complete the task faster.

### Performance Metrics in Concurrent Execution

Finally, just as we did with the sequential version, we measure the performance of this concurrent implementation:

1. **Elapsed Time**: This is the total time taken from when the first bus starts until the last bus finishes transporting all passengers. With two buses working together, we expect this time to be shorter than the sequential version.
   
2. **Throughput**: Throughput is calculated as the total number of passengers transported divided by the elapsed time. With concurrency, throughput should increase, reflecting the improved efficiency.

### Summary

By introducing concurrency gradually, we've seen how adding a second bus and using threads can significantly improve the efficiency of our passenger transportation simulation. However, with concurrency comes the need to manage shared resources carefully using locks to prevent race conditions. In the next section, we will delve deeper into the benefits and challenges of concurrent programming, as well as explore further optimizations.


In [1]:
import time
import threading
from IPython.display import clear_output

# Setup
total_passengers = 10
bus_capacity = 1
total_trips = 0
passengers_transported = 0
trip_details = []
lock = threading.Lock()

# Track the positions of each car
car_positions = {"🚗": 0, "🚙": 0}

def print_smileys(count, total_slots=10):
    smileys = "😃" * count + "⬜" * (total_slots - count)
    smiley_grid = []
    for i in range(2):  # Always print a 2x5 grid
        smiley_grid.append(smileys[i*5:(i+1)*5])
    return smiley_grid

# Transport function for a single car
def transport_passengers(car_id, car_emoji, lane, start_delay=0):
    global total_trips, passengers_transported
    time.sleep(start_delay)  # Start delay for the second car
    fixed_path_segment = "-" * 10  # Each segment between start, intermediate, and stop
    bus_paths = [
        f"{car_emoji}{fixed_path_segment}{fixed_path_segment}{fixed_path_segment}",
        f"{fixed_path_segment}{car_emoji}{fixed_path_segment}{fixed_path_segment}",
        f"{fixed_path_segment}{fixed_path_segment}{car_emoji}{fixed_path_segment}",
        f"{fixed_path_segment}{fixed_path_segment}{fixed_path_segment}{car_emoji}"
    ]

    while True:
        for path in bus_paths:
            with lock:
                if passengers_transported >= total_passengers:
                    return
                car_positions[car_emoji] = bus_paths.index(path)
                visualize_trip(total_trips, passengers_transported)
            time.sleep(0.25)
        with lock:
            if passengers_transported < total_passengers:
                total_trips += 1
                passengers_transported += bus_capacity
                if passengers_transported > total_passengers:
                    passengers_transported = total_passengers
                trip_details.append((total_trips, passengers_transported))
                car_positions[car_emoji] = len(bus_paths) - 1
                visualize_trip(total_trips, passengers_transported, final=True)
        time.sleep(0.25)  # Simulate the return of the bus
        for path in reversed(bus_paths):
            with lock:
                car_positions[car_emoji] = bus_paths.index(path)
                visualize_trip(total_trips, passengers_transported, returning=True)
            time.sleep(0.25)
        time.sleep(1)

# Visualization
def visualize_trip(trip_number, passengers_transported, final=False, returning=False):
    clear_output(wait=True)
    num_smileys_start = total_passengers - passengers_transported
    num_smileys_stop = passengers_transported
    start_smileys = print_smileys(num_smileys_start)
    stop_smileys = print_smileys(num_smileys_stop)

    fixed_path_segment = "-" * 10  # Each segment between start, intermediate, and stop
    car1_path = ["🚗" + fixed_path_segment * 3, fixed_path_segment + "🚗" + fixed_path_segment * 2,
                 fixed_path_segment * 2 + "🚗" + fixed_path_segment, fixed_path_segment * 3 + "🚗"]
    car2_path = ["🚙" + fixed_path_segment * 3, fixed_path_segment + "🚙" + fixed_path_segment * 2,
                 fixed_path_segment * 2 + "🚙" + fixed_path_segment, fixed_path_segment * 3 + "🚙"]

    car1_pos = car1_path[car_positions["🚗"]]
    car2_pos = car2_path[car_positions["🚙"]]

    print(f"Trip {trip_number}: {passengers_transported} passengers transported.")
    for row in range(2):
        if row == 0:
            print(start_smileys[row] + "  " + car1_pos + "  " + stop_smileys[row])
        else:
            print(start_smileys[row] + "  " + car2_pos + "  " + stop_smileys[row])

# Create and start threads for two cars in different lanes
car1 = threading.Thread(target=transport_passengers, args=(1, "🚗", 0))
car2 = threading.Thread(target=transport_passengers, args=(2, "🚙", 1))

start_time = time.time()
car1.start()
car2.start()

car1.join()
car2.join()

elapsed_time = time.time() - start_time
print(f"Total elapsed time: {elapsed_time:.2f} seconds")
throughput = passengers_transported / elapsed_time
print(f"Throughput: {throughput:.2f} passengers per second")
print()


Trip 10: 10 passengers transported.
⬜⬜⬜⬜⬜  🚗------------------------------  😃😃😃😃😃
⬜⬜⬜⬜⬜  🚙------------------------------  😃😃😃😃😃
Total elapsed time: 16.44 seconds
Throughput: 0.61 passengers per second



### Task: Add a Fourth Bus to the Simulation

In the following example, we simulated a transportation system using three buses, each operating concurrently to transport passengers. Now, let's extend this system by adding a fourth bus. After implementing the fourth bus, you will compare the performance (in terms of elapsed time and throughput) to the original three-bus system.

#### Step-by-Step Guide to Adding the Fourth Bus

Follow these steps to update the code and introduce a fourth bus to the simulation:

#### 1. **Update the Bus Tracking Dictionary**

First, update the `car_positions` dictionary to include the new bus:

```python
# Track the positions of each car
car_positions = {"🚗": 0, "🚙": 0, "🚐": 0, "🚌": 0}
```

#### 2. **Adjust the Visualization for Four Buses**

Next, update the `print_smileys` function to handle four rows of smileys:

```python
def print_smileys(count, total_slots=20):  # Adjusted for 20 passengers to match the grid
    smileys = "😃" * count + "⬜" * (total_slots - count)
    smiley_grid = []
    for i in range(4):  # Adjusted for a 4x5 grid
        smiley_grid.append(smileys[i*5:(i+1)*5])
    return smiley_grid
```

Also, update the `visualize_trip` function to handle the fourth bus:

```python
def visualize_trip(trip_number, passengers_transported, final=False, returning=False):
    clear_output(wait=True)
    num_smileys_start = total_passengers - passengers_transported
    num_smileys_stop = passengers_transported
    start_smileys = print_smileys(num_smileys_start, total_slots=20)  # Adjusted for more passengers
    stop_smileys = print_smileys(num_smileys_stop, total_slots=20)  # Adjusted for more passengers

    fixed_path_segment = "-" * 10  # Each segment between start, intermediate, and stop
    car1_path = ["🚗" + fixed_path_segment * 3, fixed_path_segment + "🚗" + fixed_path_segment * 2,
                 fixed_path_segment * 2 + "🚗" + fixed_path_segment, fixed_path_segment * 3 + "🚗"]
    car2_path = ["🚙" + fixed_path_segment * 3, fixed_path_segment + "🚙" + fixed path_segment * 2,
                 fixed_path_segment * 2 + "🚙" + fixed_path_segment, fixed_path_segment * 3 + "🚙"]
    car3_path = ["🚐" + fixed_path_segment * 3, fixed_path_segment + "🚐" + fixed path_segment * 2,
                 fixed_path_segment * 2 + "🚐" + fixed path_segment, fixed path_segment * 3 + "🚐"]
    car4_path = ["🚌" + fixed path_segment * 3, fixed path segment + "🚌" + fixed path segment * 2,
                 fixed path segment * 2 + "🚌" + fixed path segment, fixed path segment * 3 + "🚌"]

    car1_pos = car1_path[car_positions["🚗"]]
    car2_pos = car2_path[car_positions["🚙"]]
    car3_pos = car3_path[car_positions["🚐"]]
    car4_pos = car4_path[car_positions["🚌"]]

    print(f"Trip {trip_number}: {passengers transported} passengers transported.")
    for row in range(4):  # Adjusted to print 4 rows
        if row == 0:
            print(start_smileys[row] + "  " + car1_pos + "  " + stop_smileys[row])
        elif row == 1:
            print(start_smileys[row] + "  " + car2_pos + "  " + stop_smileys[row])
        elif row == 2:
            print(start_smileys[row] + "  " + car3_pos + "  " + stop_smileys[row])
        elif row == 3:
            print(start_smileys[row] + "  " + car4_pos + "  " + stop_smileys[row])

```

#### 3. **Create and Start a Thread for the Fourth Bus**

Now, create and start a new thread for the fourth bus, `🚌`:

```python
# Create and start threads for four cars in different lanes
car1 = threading.Thread(target=transport_passengers, args=("🚗", 0, 0))
car2 = threading.Thread(target=transport_passengers, args=("🚙", 1, 0.5))
car3 = threading.Thread(target=transport_passengers, args=("🚐", 2, 1))
car4 = threading.Thread(target=transport_passengers, args=("🚌", 3, 1.5))  # New bus with a start delay

start_time = time.time()
car1.start()
car2.start()
car3.start()
car4.start()  # Start the fourth bus

car1.join()
car2.join()
car3.join()
car4.join()  # Wait for the fourth bus to finish

elapsed_time = time.time() - start_time
print(f"Total elapsed time: {elapsed_time:.2f} seconds")
throughput = passengers_transported / elapsed_time
print(f"Throughput: {throughput:.2f} passengers per second")

```

#### 4. **Run and Compare the Performance**
After making the changes, run the updated code. Observe the `elapsed_time` and `throughput` values, and compare them to the original three-bus implementation. You should notice that with the fourth bus, the system can transport passengers more quickly, reducing the total time and increasing throughput.

### **Reflection Questions**
1. What differences did you observe in performance when adding the fourth bus?
2. Did the throughput increase as expected? Why or why not?
3. How does adding more buses impact the overall system? Are there any diminishing returns as you keep adding buses?
4. What challenges might arise if you were to continue scaling this system with even more buses?

In [ ]:
import time
import threading
from IPython.display import clear_output

# Setup
total_passengers = 10  # Adjusted for more passengers since we have 3 cars
bus_capacity = 1
total_trips = 0
passengers_transported = 0
trip_details = []
lock = threading.Lock()

# Track the positions of each car
car_positions = {"🚗": 0, "🚙": 0, "🚐": 0}

def print_smileys(count, total_slots=15):  # Adjusted for 15 passengers to match the grid
    smileys = "😃" * count + "⬜" * (total_slots - count)
    smiley_grid = []
    for i in range(3):  # Adjusted for a 3x5 grid
        smiley_grid.append(smileys[i*5:(i+1)*5])
    return smiley_grid

# Transport function for a single car
def transport_passengers(car_emoji, lane, start_delay=0):
    global total_trips, passengers_transported
    time.sleep(start_delay)  # Start delay for subsequent cars
    fixed_path_segment = "-" * 10  # Each segment between start, intermediate, and stop
    bus_paths = [
        f"{car_emoji}{fixed_path_segment * 3}",
        f"{fixed_path_segment}{car_emoji}{fixed_path_segment * 2}",
        f"{fixed_path_segment * 2}{car_emoji}{fixed_path_segment}",
        f"{fixed_path_segment * 3}{car_emoji}"
    ]

    while True:
        for path in bus_paths:
            with lock:
                if passengers_transported >= total_passengers:
                    return
                car_positions[car_emoji] = bus_paths.index(path)
                visualize_trip(total_trips, passengers_transported)
            time.sleep(0.25)
        with lock:
            if passengers_transported < total_passengers:
                total_trips += 1
                passengers_transported += bus_capacity
                if passengers_transported > total_passengers:
                    passengers_transported = total_passengers
                trip_details.append((total_trips, passengers_transported))
                car_positions[car_emoji] = len(bus_paths) - 1
                visualize_trip(total_trips, passengers_transported, final=True)
        time.sleep(0.25)  # Simulate the return of the bus
        for path in reversed(bus_paths):
            with lock:
                car_positions[car_emoji] = bus_paths.index(path)
                visualize_trip(total_trips, passengers_transported, returning=True)
            time.sleep(0.25)
        time.sleep(1)

# Visualization
def visualize_trip(trip_number, passengers_transported, final=False, returning=False):
    clear_output(wait=True)
    num_smileys_start = total_passengers - passengers_transported
    num_smileys_stop = passengers_transported
    start_smileys = print_smileys(num_smileys_start, total_slots=15)  # Adjusted for more passengers
    stop_smileys = print_smileys(num_smileys_stop, total_slots=15)  # Adjusted for more passengers

    fixed_path_segment = "-" * 10  # Each segment between start, intermediate, and stop
    car1_path = ["🚗" + fixed_path_segment * 3, fixed_path_segment + "🚗" + fixed_path_segment * 2,
                 fixed_path_segment * 2 + "🚗" + fixed_path_segment, fixed_path_segment * 3 + "🚗"]
    car2_path = ["🚙" + fixed_path_segment * 3, fixed_path_segment + "🚙" + fixed_path_segment * 2,
                 fixed_path_segment * 2 + "🚙" + fixed_path_segment, fixed_path_segment * 3 + "🚙"]
    car3_path = ["🚐" + fixed_path_segment * 3, fixed_path_segment + "🚐" + fixed_path_segment * 2,
                 fixed_path_segment * 2 + "🚐" + fixed_path_segment, fixed_path_segment * 3 + "🚐"]

    car1_pos = car1_path[car_positions["🚗"]]
    car2_pos = car2_path[car_positions["🚙"]]
    car3_pos = car3_path[car_positions["🚐"]]

    print(f"Trip {trip_number}: {passengers_transported} passengers transported.")
    for row in range(3):
        if row == 0:
            print(start_smileys[row] + "  " + car1_pos + "  " + stop_smileys[row])
        elif row == 1:
            print(start_smileys[row] + "  " + car2_pos + "  " + stop_smileys[row])
        elif row == 2:
            print(start_smileys[row] + "  " + car3_pos + "  " + stop_smileys[row])

# Create and start threads for three cars in different lanes
car1 = threading.Thread(target=transport_passengers, args=("🚗", 0, 0))
car2 = threading.Thread(target=transport_passengers, args=("🚙", 1, 0.5))
car3 = threading.Thread(target=transport_passengers, args=("🚐", 2, 1))

start_time = time.time()
car1.start()
car2.start()
car3.start()

car1.join()
car2.join()
car3.join()

elapsed_time = time.time() - start_time
print(f"Total elapsed time: {elapsed_time:.2f} seconds")
throughput = passengers_transported / elapsed_time
print(f"Throughput: {throughput:.2f} passengers per second")
print()


Trip 10: 10 passengers transported.
⬜⬜⬜⬜⬜  🚗------------------------------  😃😃😃😃😃
⬜⬜⬜⬜⬜  --------------------🚙----------  😃😃😃😃😃
⬜⬜⬜⬜⬜  🚐------------------------------  ⬜⬜⬜⬜⬜
Total elapsed time: 13.12 seconds
Throughput: 0.76 passengers per second



In [ ]:
import time
from IPython.display import clear_output

# Setup
total_passengers = 10
bus_capacity = 1
total_trips = 0
passengers_transported = 0
trip_details = []

# Track the position of the car
car_position = 0

# Track waiting start times of passengers
waiting_start_times = [[None for _ in range(5)] for _ in range(3)]  # 3x5 matrix with initial None
boarding_times = []

def initialize_waiting_times():
    current_time = time.time()
    for row in range(3):
        for col in range(5):
            waiting_start_times[row][col] = current_time

def print_smileys(count, total_slots=10):
    smileys = "😃" * count + "⬜" * (total_slots - count)
    smiley_grid = []
    for i in range(2):  # Always print a 2x5 grid
        smiley_grid.append(smileys[i*5:(i+1)*5])
    return smiley_grid

# Transport function for a single car
def transport_passengers(car_emoji, lane):
    global total_trips, passengers_transported, car_position
    fixed_path_segment = "-" * 10  # Each segment between start, intermediate, and stop
    bus_paths = [
        f"{car_emoji}{fixed_path_segment}{fixed_path_segment}{fixed_path_segment}",
        f"{fixed_path_segment}{car_emoji}{fixed_path_segment}{fixed_path_segment}",
        f"{fixed_path_segment}{fixed_path_segment}{car_emoji}{fixed_path_segment}",
        f"{fixed_path_segment}{fixed_path_segment}{fixed_path_segment}{car_emoji}"
    ]

    while True:
        for path in bus_paths:
            if passengers_transported >= total_passengers:
                return
            car_position = bus_paths.index(path)
            visualize_trip(total_trips, passengers_transported)
            time.sleep(0.25)
        if passengers_transported < total_passengers:
            total_trips += 1
            row, col = divmod(passengers_transported, 5)  # Determine the passenger position in the matrix
            passengers_transported += bus_capacity
            if passengers_transported > total_passengers:
                passengers_transported = total_passengers
            trip_details.append((total_trips, passengers_transported))
            car_position = len(bus_paths) - 1
            visualize_trip(total_trips, passengers_transported, final=True)
            boarding_times.append((time.time(), row, col))

        time.sleep(0.25)  # Simulate the return of the bus
        for path in reversed(bus_paths):
            car_position = bus_paths.index(path)
            visualize_trip(total_trips, passengers_transported, returning=True)

            time.sleep(0.25)
        time.sleep(1)

# Calculate and print average waiting time
def print_average_waiting_time():
    total_waiting_time = 0
    count = len(boarding_times)
    for boarding_time, row, col in boarding_times:
        if waiting_start_times[row][col] is not None:
            total_waiting_time += boarding_time - waiting_start_times[row][col]
    average_waiting_time = total_waiting_time / count if count > 0 else 0
    print(f"Average waiting time: {average_waiting_time:.2f} seconds")

# Visualization
def visualize_trip(trip_number, passengers_transported, final=False, returning=False):
    clear_output(wait=True)
    num_smileys_start = total_passengers - passengers_transported
    num_smileys_stop = passengers_transported
    start_smileys = print_smileys(num_smileys_start)
    stop_smileys = print_smileys(num_smileys_stop)

    fixed_path_segment = "-" * 10  # Each segment between start, intermediate, and stop
    car_path = ["🚗" + fixed_path_segment * 3, fixed_path_segment + "🚗" + fixed_path_segment * 2,
                fixed_path_segment * 2 + "🚗" + fixed_path_segment, fixed_path_segment * 3 + "🚗"]

    car_pos = car_path[car_position]

    print(f"Trip {trip_number}: {passengers_transported} passengers transported.")
    for row in range(2):
        if row == 0:
            print(start_smileys[row] + "  " + car_pos + "  " + stop_smileys[row])
        else:
            print(start_smileys[row] + "  " + " " * len(car_pos) + "  " + stop_smileys[row])

# Run the transport function
initialize_waiting_times()
start_time = time.time()
transport_passengers("🚗", 0)
elapsed_time = time.time() - start_time
print(f"Total elapsed time: {elapsed_time:.2f} seconds")
throughput = passengers_transported / elapsed_time
print(f"Throughput: {throughput:.2f} passengers per second")
print_average_waiting_time()
print()


Trip 10: 10 passengers transported.
⬜⬜⬜⬜⬜  🚗------------------------------  😃😃😃😃😃
⬜⬜⬜⬜⬜                                   😃😃😃😃😃
Total elapsed time: 32.72 seconds
Throughput: 0.31 passengers per second
Average waiting time: 15.74 seconds



In [ ]:
import time
import threading
from IPython.display import clear_output

# Setup
total_passengers = 10  # Adjusted for more passengers since we have 3 cars
bus_capacity = 1
total_trips = 0
passengers_transported = 0
trip_details = []
lock = threading.Lock()

# Track the positions of each car
car_positions = {"🚗": 0, "🚙": 0, "🚐": 0}

# Track waiting start times of passengers
waiting_start_times = [[None for _ in range(5)] for _ in range(3)]  # 3x5 matrix with initial None
boarding_times = []

def initialize_waiting_times():
    current_time = time.time()
    for row in range(3):
        for col in range(5):
            waiting_start_times[row][col] = current_time

def print_smileys(count, total_slots=15):  # Adjusted for 15 passengers to match the grid
    smileys = "😃" * count + "⬜" * (total_slots - count)
    smiley_grid = []
    for i in range(3):  # Adjusted for a 3x5 grid
        smiley_grid.append(smileys[i*5:(i+1)*5])
    return smiley_grid

# Transport function for a single car
def transport_passengers(car_emoji, lane, start_delay=0):
    global total_trips, passengers_transported
    time.sleep(start_delay)  # Start delay for subsequent cars
    fixed_path_segment = "-" * 10  # Each segment between start, intermediate, and stop
    bus_paths = [
        f"{car_emoji}{fixed_path_segment * 3}",
        f"{fixed_path_segment}{car_emoji}{fixed_path_segment * 2}",
        f"{fixed_path_segment * 2}{car_emoji}{fixed_path_segment}",
        f"{fixed_path_segment * 3}{car_emoji}"
    ]

    while True:
        for path in bus_paths:
            with lock:
                if passengers_transported >= total_passengers:
                    return
                car_positions[car_emoji] = bus_paths.index(path)
                visualize_trip(total_trips, passengers_transported)

            time.sleep(0.25)
        with lock:
            if passengers_transported < total_passengers:
                total_trips += 1
                row, col = divmod(passengers_transported, 5)  # Determine the passenger position in the matrix
                passengers_transported += bus_capacity
                if passengers_transported > total_passengers:
                    passengers_transported = total_passengers
                trip_details.append((total_trips, passengers_transported))
                car_positions[car_emoji] = len(bus_paths) - 1
                visualize_trip(total_trips, passengers_transported, final=True)
                boarding_times.append((time.time(), row, col))
        time.sleep(0.25)  # Simulate the return of the bus
        for path in reversed(bus_paths):
            with lock:
                car_positions[car_emoji] = bus_paths.index(path)
                visualize_trip(total_trips, passengers_transported, returning=True)

            time.sleep(0.25)
        time.sleep(1)

# Calculate and print average waiting time
def print_average_waiting_time():
    total_waiting_time = 0
    count = len(boarding_times)
    for boarding_time, row, col in boarding_times:
        if waiting_start_times[row][col] is not None:
            total_waiting_time += boarding_time - waiting_start_times[row][col]
    average_waiting_time = total_waiting_time / count if count > 0 else 0
    print(f"Average waiting time: {average_waiting_time:.2f} seconds")


# Visualization
def visualize_trip(trip_number, passengers_transported, final=False, returning=False):
    clear_output(wait=True)
    num_smileys_start = total_passengers - passengers_transported
    num_smileys_stop = passengers_transported
    start_smileys = print_smileys(num_smileys_start, total_slots=15)  # Adjusted for more passengers
    stop_smileys = print_smileys(num_smileys_stop, total_slots=15)  # Adjusted for more passengers

    fixed_path_segment = "-" * 10  # Each segment between start, intermediate, and stop
    car1_path = ["🚗" + fixed_path_segment * 3, fixed_path_segment + "🚗" + fixed_path_segment * 2,
                 fixed_path_segment * 2 + "🚗" + fixed_path_segment, fixed_path_segment * 3 + "🚗"]
    car2_path = ["🚙" + fixed_path_segment * 3, fixed_path_segment + "🚙" + fixed_path_segment * 2,
                 fixed_path_segment * 2 + "🚙" + fixed_path_segment, fixed_path_segment * 3 + "🚙"]
    car3_path = ["🚐" + fixed_path_segment * 3, fixed_path_segment + "🚐" + fixed_path_segment * 2,
                 fixed_path_segment * 2 + "🚐" + fixed_path_segment, fixed_path_segment * 3 + "🚐"]

    car1_pos = car1_path[car_positions["🚗"]]
    car2_pos = car2_path[car_positions["🚙"]]
    car3_pos = car3_path[car_positions["🚐"]]

    print(f"Trip {trip_number}: {passengers_transported} passengers transported.")
    for row in range(3):
        if row == 0:
            print(start_smileys[row] + "  " + car1_pos + "  " + stop_smileys[row])
        elif row == 1:
            print(start_smileys[row] + "  " + car2_pos + "  " + stop_smileys[row])
        elif row == 2:
            print(start_smileys[row] + "  " + car3_pos + "  " + stop_smileys[row])
    elapsed_time = time.time() - start_time
    throughput = passengers_transported / elapsed_time
    print(f"Throughput: {throughput:.2f} passengers per second")
    print()

# Create and start threads for three cars in different lanes
def start_simulation():
    initialize_waiting_times()
    car1 = threading.Thread(target=transport_passengers, args=("🚗", 0, 0))
    car2 = threading.Thread(target=transport_passengers, args=("🚙", 1, 0.5))
    car3 = threading.Thread(target=transport_passengers, args=("🚐", 2, 1))

    global start_time
    start_time = time.time()
    car1.start()
    car2.start()
    car3.start()

    car1.join()
    car2.join()
    car3.join()

    elapsed_time = time.time() - start_time
    print(f"Total elapsed time: {elapsed_time:.2f} seconds")
    throughput = passengers_transported / elapsed_time
    print(f"Throughput: {throughput:.2f} passengers per second")
    print_average_waiting_time()
    print()

# Start the simulation
start_simulation()


Trip 10: 10 passengers transported.
⬜⬜⬜⬜⬜  🚗------------------------------  😃😃😃😃😃
⬜⬜⬜⬜⬜  --------------------🚙----------  😃😃😃😃😃
⬜⬜⬜⬜⬜  🚐------------------------------  ⬜⬜⬜⬜⬜
Throughput: 0.84 passengers per second

Total elapsed time: 13.15 seconds
Throughput: 0.76 passengers per second
Average waiting time: 5.41 seconds



In [ ]:
import time
import threading
import random
from IPython.display import clear_output

# Setup
total_passengers = 200  # Number of passengers
counter = 0
lock = threading.Lock()

# Function that increments the counter
def increment_counter(passenger_id):
    global counter
    # Uncomment the next line to introduce a lock and prevent race condition
    #with lock:
    # Simulate some work with random delay
    local_counter = counter
    time.sleep(random.uniform(0.001, 0.01))  # Artificial random delay to increase the chance of race condition
    local_counter += 1
    counter = local_counter
    visualize(passenger_id)

# Visualization function
def visualize(passenger_id):
    clear_output(wait=True)
    print(f"Passenger {passenger_id} incremented the counter.")
    print(f"Current counter value: {counter}")
    print("Passengers who have incremented the counter:")
    for i in range(total_passengers):
        if i <= passenger_id:
            print("😃", end=" ")
        else:
            print("⬜", end=" ")
    print()

# Function that simulates a passenger incrementing the counter
def passenger_thread(passenger_id):
    time.sleep(0.05 * passenger_id)  # Stagger the passengers a bit
    increment_counter(passenger_id)

# Create and start threads for each passenger
threads = []
for i in range(total_passengers):
    t = threading.Thread(target=passenger_thread, args=(i,))
    threads.append(t)
    t.start()

# Wait for all threads to finish
for t in threads:
    t.join()

print("All passengers have incremented the counter.")
print(f"Final counter value: {counter}")


Passenger 199 incremented the counter.
Current counter value: 200
Passengers who have incremented the counter:
😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 
All passengers have incremented the counter.
Final counter value: 200
